In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle
import json
from Utils.evaluation import EvaluationMetrics, ActualStats
from Utils.helper import *
from Utils.sample_squads import *
import matplotlib
matplotlib.rcParams.update({'font.size': 22,
                            'font.weight': 'bold'})

## History loss plot for both models

In [ ]:
innings_number = 1
with open("Models\Inn1-HeavyDense-ep0to20\history.pkl", "rb") as fp:
    history1 = pickle.load(fp)
with open("Models\Inn1-HeavyDense-ep20to50\history.pkl", "rb") as fp:
    history2 = pickle.load(fp)
history = {}
for key in history1:
    history[key] = history1[key] + history2[key]
for key in sorted([i for i in history if i.split("_")[0]!="val"]):
    epochs = len(history[key])
    fig = plt.gcf()
    fig.set_size_inches(10, 7)
    # plt.figure(dpi=300)
    plt.plot([i for i in range(1, epochs+1)], history[key])
    plt.plot([i for i in range(1, epochs+1)], history["val_"+key])
    plt.xlabel('epoch', fontweight='bold')
    plt.ylabel(key, fontweight='bold')
    plt.legend(["Training", "Validation"])
    plt.title(f"Innings {innings_number} - {key}", fontweight='bold')
    plt.savefig(f"Images\\Training\\inn{innings_number}_{key}.jpeg", dpi=300)
    plt.show()

In [ ]:
innings_number = 2
with open("Models\Inn2-HeavyDenseRequired-ep0to20\history.pkl", "rb") as fp:
    history1 = pickle.load(fp)
with open("Models\Inn2-HeavyDenseRequired-ep20to30\history.pkl", "rb") as fp:
    history2 = pickle.load(fp)
with open("Models\Inn2-HeavyDenseRequired-ep30to50\history.pkl", "rb") as fp:
    history3 = pickle.load(fp)
history = {}
for key in history1:
    history[key] = history1[key] + history2[key] + history3[key]
for key in sorted([i for i in history if i.split("_")[0]!="val"]):
    epochs = len(history[key])
    fig = plt.gcf()
    fig.set_size_inches(10, 7)
    # plt.figure(dpi=300)
    plt.plot([i for i in range(1, epochs+1)], history[key])
    plt.plot([i for i in range(1, epochs+1)], history["val_"+key])
    plt.xlabel('epoch', fontweight='bold')
    plt.ylabel(key, fontweight='bold')
    plt.legend(["Training", "Validation"])
    plt.title(f"Innings {innings_number} - {key}", fontweight='bold')
    plt.savefig(f"Images\\Training\\inn{innings_number}_{key}.jpeg", dpi=300)
    plt.show()

## Initialize

In [ ]:
model_inn1 = tf.keras.models.load_model('Models\Inn1-HeavyDense-ep20to50\cp-0029.h5')
model_inn2 = tf.keras.models.load_model('Models\Inn2-HeavyDenseRequired-ep30to50\cp-0016.h5')

In [ ]:
evaluator = EvaluationMetrics(model_inn1, model_inn2,
                              # )
                              "Evaluation/tournament3_1.pkl")

In [ ]:
# set_bs = set(BS_Cols)
# for i in evaluator.teams:
#     (batting, bowling), venue = i
#     for name in batting[1:]:
#         if "Striker_"+name not in set_bs:
#             print("Striker_"+name)
#         if "Non_Striker_"+name not in set_bs:
#             print("Non_Striker_"+name)
#         if "Bowler_"+name not in set_bs:
#             print("Bowler_"+name)
#     for name in bowling[1:]:
#         if "Striker_"+name not in set_bs:
#             print("Striker_"+name)
#         if "Non_Striker_"+name not in set_bs:
#             print("Non_Striker_"+name)
#         if "Bowler_"+name not in set_bs:
#             print("Bowler_"+name)
#     if "Venue_"+venue not in set_bs:
#         print("Venue_"+venue)
#     if "Batting_Team_"+batting[0] not in set_bs:
#         print("Batting_Team_"+batting[0])
#     if "Batting_Team_"+bowling[0] not in set_bs:
#         print("Batting_Team_"+bowling[0])
#     if "Toss_"+batting[0] not in set_bs:
#         print("Toss_"+batting[0])
#     if "Toss_"+bowling[0] not in set_bs:
#         print("Toss_"+bowling[0])
#     if "Bowling_Team_"+batting[0] not in set_bs:
#         print("Bowling_Team_"+batting[0])
#     if "Bowling_Team_"+bowling[0] not in set_bs:
#         print("Bowling_Team_"+bowling[0])

In [ ]:
# num_tournaments = 5
# verbose = 0
# for t in range(num_tournaments):
#     print(f"Tournament {t+1}/{num_tournaments}:")
#     evaluator.reinitialize_tournament()
#     for _ in tqdm(range(56), ncols=80, disable=True if verbose!=0 else False):
#         if verbose != 0:
#             print(f"\n\nMatch number: {evaluator.match_count+1}")   
#         evaluator.simulate_match(verbose)   
#     evaluator.display_table()
#     evaluator.save_object("Evaluation/tournament3_1.pkl")

In [ ]:
actualstat = ActualStats("Evaluation/actual.pkl")
# actualstat = ActualStats()
# actualstat.run_df(1, True)
# actualstat.run_df(2, True)
# actualstat.save_object("Evaluation/actual.pkl")   

In [ ]:
len(actualstat.chasing_stat), len(actualstat.total_stat)

## First Innings Target Evaluation

In [ ]:
from scipy import stats
# stats.ttest_ind(evaluator.total_stat[::2], actualstat.total_stat, equal_var=False)
stats.ks_2samp(evaluator.total_stat, actualstat.total_stat)

In [ ]:
len(evaluator.total_stat), len(actualstat.total_stat)

In [ ]:
import matplotlib.pyplot as plt
plt.hist([evaluator.total_stat, actualstat.total_stat], density=True, bins=10, label=["Our Simulation", "Actual Matches"])
plt.legend()
plt.xlabel("First Innings Total", fontweight='bold')
plt.ylabel("Relative Frequency", fontweight='bold')
fig = plt.gcf()
fig.set_size_inches(15, 8)
plt.savefig("Images\\Evaluation\\inn1_total_histogram.jpeg", dpi=300)
plt.show()

In [ ]:
import statistics
import math
mean_sim = statistics.mean(evaluator.total_stat)
variance_sim = statistics.variance(evaluator.total_stat)
stdev_sim = statistics.stdev(evaluator.total_stat)
mean_actual = statistics.mean(actualstat.total_stat)
variance_actual = statistics.variance(actualstat.total_stat)
stdev_actual = statistics.stdev(actualstat.total_stat)
z = abs(mean_actual - mean_sim)/(math.sqrt(variance_actual+variance_sim))
print(z)

In [ ]:
mean_sim, mean_actual, stdev_sim, stdev_actual, 

## Second Innings Evaluation

In [ ]:
actual_chasing_stat = actualstat.chasing_stat
# evaluator_chasing_stat = evaluator.chasing_stat
# actual_chasing_stat
evaluator_chasing_stat = []
for inn1, inn2 in evaluator.innings_obj_list:
    row_dic = {}
    row_dic["Final_Score"] = [inn2.Runs, inn2.Wickets]
    row_dic["First_Innings_Score"] = inn1.Runs
    row_dic["Overs"] = [inn2.Overs - 1, inn2.Balls - 1]
    row_dic["Chasing_Team"] = inn2.Batting_Team
    row_dic["Defending_Team"] = inn2.Bowling_Team
    final_score = inn2.Runs
    first_innings_score = inn1.Runs
    if final_score > first_innings_score:
            outcome = 1
    elif final_score == first_innings_score:
        outcome = 0
    elif final_score < first_innings_score:
        outcome = -1
    else:
        assert False, "Wrong if conditions"
    row_dic["Outcome"] = outcome
    evaluator_chasing_stat.append(row_dic)

In [ ]:
actual_chasing_outcome = [i["Outcome"] for i in actual_chasing_stat]
evaluator_chasing_outcome = [i["Outcome"] for i in evaluator_chasing_stat]
pie_lis = []
labels = ["Chasing Team Lost", "Tie", "Chasing Team Won"]
for outcome_lis in [evaluator_chasing_outcome, actual_chasing_outcome]:
    res_dist = {}
    for res in (outcome_lis):
        if res in res_dist:
            res_dist[res] += 1
        else:
            res_dist[res] = 1
    pie_lis.append([res_dist[i] for i in sorted(res_dist)])

fig = plt.gcf()
fig.set_size_inches(10, 10)
plt.pie(pie_lis[0], labels=labels, autopct="%.2f")
plt.legend(title = "Our Simulation")
plt.savefig("Images\\Evaluation\\inn2_chasing_dist_simulation.jpeg", dpi=300)
plt.show()
fig = plt.gcf()
fig.set_size_inches(10, 10)
plt.pie(pie_lis[1], labels=labels, autopct="%.2f")
plt.legend(title = "Actual Matches", loc="lower right")
plt.savefig("Images\\Evaluation\\inn2_chasing_dist_actual.jpeg", dpi=300)
plt.show()

In [ ]:
step = 50
thresh = 1
def my_autopct(pct):
    return ('%.2f' % pct) if pct > thresh else ''
for inn1_start, inn1_end in [[i+1, i+step] for i in range(50, 250, step)]:
    actual_chasing_outcome = [i["Outcome"] for i in actual_chasing_stat 
                            if (inn1_start <= i["First_Innings_Score"] <= inn1_end)]
    evaluator_chasing_outcome = [i["Outcome"] for i in evaluator_chasing_stat
                                if (inn1_start <= i["First_Innings_Score"] <= inn1_end)]
    pie_lis = []
    labels = ["Chasing Team Lost", "Tie", "Chasing Team Won"]
    for outcome_lis in [evaluator_chasing_outcome, actual_chasing_outcome]:
        res_dist = { -1: 0, 0: 0, 1: 0}
        for res in (outcome_lis):
            if res in res_dist:
                res_dist[res] += 1
            else:
                res_dist[res] = 1
        pie_lis.append([res_dist[i] for i in sorted(res_dist)])

    fig = plt.gcf()
    fig.set_size_inches(14, 14)
    labels_new = [labels[i] if pie_lis[0][i]/sum(pie_lis[0])*100 >= thresh else ''
          for i in range(len(labels))]

    legend_loc = "upper right"
    if inn1_end == 150:
        legend_loc = "lower right"
    
    plt.pie(pie_lis[0], labels=[""]*len(labels_new), autopct=my_autopct)
    plt.legend(labels=labels, title="Match Outcome", loc=legend_loc)
    plt.title(f"Our Simulation target \nfrom {inn1_start} to {inn1_end}", fontweight='bold')
    plt.savefig(f"Images\\Evaluation\\inn2_chasing_dist_{inn1_start}to{inn1_end}_simulation.jpeg", dpi=300)
    plt.show()
    fig = plt.gcf()
    fig.set_size_inches(14, 14)
    labels_new = [labels[i] if pie_lis[1][i]/sum(pie_lis[1])*100 >= thresh else ''
          for i in range(len(labels))]
    plt.pie(pie_lis[1], labels=[""]*len(labels_new), autopct=my_autopct)
    plt.legend(labels=labels, title=f"Match Outcome", loc=legend_loc)
    plt.title(f"Actual Matches  target \nfrom {inn1_start} to {inn1_end}", fontweight='bold')
    plt.savefig(f"Images\\Evaluation\\inn2_chasing_dist_{inn1_start}to{inn1_end}_actual.jpeg", dpi=300)
    plt.show()

In [ ]:
teams = set(evaluator.season_table.keys())
labels = ["Loose", "Tie", "Win"]
team_outcome_dic = [{team: [0, 0, 0] for team in teams}, {team: [0, 0, 0] for team in teams}]
for ind, val in enumerate([evaluator_chasing_stat, actual_chasing_stat]):
    for match_stat in val:
        if match_stat["Chasing_Team"] not in teams or match_stat["Defending_Team"] not in teams:
            continue
        if outcome == -1:
            team_outcome_dic[ind][match_stat["Chasing_Team"]][0] += 1
            team_outcome_dic[ind][match_stat["Defending_Team"]][2] += 1
        if outcome == 0:
            team_outcome_dic[ind][match_stat["Chasing_Team"]][1] += 1
            team_outcome_dic[ind][match_stat["Defending_Team"]][1] += 1
        if outcome == 1:
            team_outcome_dic[ind][match_stat["Chasing_Team"]][2] += 1
            team_outcome_dic[ind][match_stat["Defending_Team"]][1] += 1

In [ ]:
# for team in teams:
#     fig = plt.gcf()
#     fig.set_size_inches(8, 8)
#     plt.pie(team_outcome_dic[0][team], labels=labels, autopct="%.2f")
#     plt.legend(title = f"Our Simulation: {team}")
#     plt.show()
#     fig = plt.gcf()
#     fig.set_size_inches(8, 8)
#     plt.pie(team_outcome_dic[1][team], labels=labels, autopct="%.2f")
#     plt.legend(title = f"Actual Match: {team}")
#     plt.show()

## Progression Evaluation

In [ ]:
intervals = ((1, 6), (7, 10), (11, 15), (16, 20))
# actualstat = ActualStats(intervals=intervals)
# actualstat.run_df(1, True)
# actualstat.run_df(2, True)
over_to_interval = {}
for interval in intervals:
    for i in range(interval[0], interval[1]+1):
        over_to_interval[i] = interval
new_progress_stat = {"runs": {i:[] for i in intervals},
                     "wickets": {i:[] for i in intervals},}
for inn1, inn2 in (evaluator.innings_obj_list):
    for inn in [inn1, inn2]:
        innings_progress = {"runs": {i:0 for i in intervals},
                            "wickets": {i:0 for i in intervals},}
        for ind, over_summary in enumerate(inn.Overs_Summary):
            o = ind+1
            innings_progress["runs"][over_to_interval[o]] += over_summary[0]
            innings_progress["wickets"][over_to_interval[o]] += over_summary[1]
        for key in innings_progress:
            for interval in intervals:
                if o < interval[1]:
                    continue
                new_progress_stat[key][interval].append(innings_progress[key][interval])

In [ ]:
import matplotlib.pyplot as plt
bins = {"runs": 15, "wickets": 20}
scored_taken = {"runs": "scored", "wickets": "taken"}
for key in ["runs", "wickets"]:
    for interval in (intervals):
        plt.hist([new_progress_stat[key][interval], actualstat.new_progression_stat[key][interval]],
                 density=True, bins=bins[key], label=["Our Simulation", "Actual Matches"])
        plt.legend()
        plt.xlabel(f"{key.title()} {scored_taken[key]} in {interval[0]}-{interval[1]}", fontweight='bold')
        plt.ylabel("Relative Frequency", fontweight='bold')
        fig = plt.gcf()
        fig.subplots_adjust(bottom=0.15)
        fig.subplots_adjust(left=0.15)
        fig.set_size_inches(10, 5)
        plt.savefig(f"Images\\Evaluation\\progress_eval_{key}_{interval[0]}to{interval[1]}_histogram.jpeg", dpi=300)
        plt.show()

In [ ]:
print(len(actualstat.new_progression_stat["runs"][intervals[0]]),len(actualstat.new_progression_stat["runs"][intervals[1]]),len(actualstat.new_progression_stat["runs"][intervals[2]]),len(actualstat.new_progression_stat["runs"][intervals[3]]))
print(len(new_progress_stat["runs"][intervals[0]]),len(new_progress_stat["runs"][intervals[1]]),len(new_progress_stat["runs"][intervals[2]]),len(new_progress_stat["runs"][intervals[3]]))

In [ ]:
# import matplotlib.pyplot as plt
# for key in ["runs", "wickets"]:
#     for i in range(4):
#         plt.hist([[k for k in evaluator.progression_stat[key][i] if k!=0], [k for k in actualstat.progression_stat[key][i] if k!=0]],
#                  density=True, bins=10, label=["Our Simulation", "Actual Matches"])
#         plt.legend()
#         plt.xlabel(f"{key.title()} scored in {i*5+1}-{i*5+5}")
#         plt.ylabel("Relative Frequency")
#         fig = plt.gcf()
#         fig.set_size_inches(8, 4)
#         plt.show()

## Batsmen Statistics

In [ ]:
batsmen_result_dic = {}
bat_stat = evaluator.batsmen_stat
for batsman in bat_stat:
    tot_runs = 0
    tot_balls = 0
    num_4 = 0
    num_6 = 0
    num_inn = 0
    num_out = 0
    hs = [0, ""]
    dismissal_type = []
    dismissed_by = []
    for i in bat_stat[batsman]:
        num_inn += 1
        tot_runs += i["Runs"]
        if i["Runs"] > hs[0] or (i["Runs"] == hs[0] and hs[1] == ""):
            hs[0] = i["Runs"]
            if i["Dismissal Type"] != "Not Out":
                hs[1] = "*"
        tot_balls += i["Balls Faced"]
        num_4 += i["Fours"]
        num_6 += i["Sixes"]
        if i["Dismissal Type"] != "Not Out":
            num_out += 1
        dismissal_type.append(i["Dismissal Type"])
        dismissed_by.append(i["Dismissed By"])
    batsmen_result_dic[batsman] = {
        "Innings": num_inn,
        "Runs": tot_runs,
        "Balls": tot_balls,
        "Fours": num_4,
        "Sixes": num_6,
        "High Score": "".join([str(i) for i in hs]),
        "Average": tot_runs/num_out if num_out!=0 else tot_runs,
        "Strike Rate": tot_runs/tot_balls*100 if tot_balls!=0 else 0,
        "Boundry Percent": (6*num_6+4*num_4)/tot_runs*100 if tot_runs!=0 else 0,
        }
batsmen_stat_df = pd.DataFrame.from_dict(batsmen_result_dic, orient='index')
save_df = batsmen_stat_df.sort_values(by=["Runs", "Balls"], ascending=False)
save_df.to_csv("Evaluation/Evaluate_Batsmen.csv")
save_df.head(20)

## Bowler Statistics

In [ ]:
bowler_result_dic = {}
bowler_stat = evaluator.bowler_stat
for bowler in bowler_stat:
    tot_runs = 0
    tot_balls = 0
    tot_wickets = 0
    for i in bowler_stat[bowler]:
        tot_runs += i["Runs Conceded"]
        tot_balls += i["Balls"]
        tot_wickets += i["Wickets Taken"]
    bowler_result_dic[bowler] = {
        "Runs Conceded": tot_runs,
        "Overs": f"{tot_balls//6}.{tot_balls%6}",
        "Wickets": tot_wickets,
        "Economy": tot_runs/tot_balls*6 if tot_balls != 0 else 0,
        }
bowler_stat_df = pd.DataFrame.from_dict(bowler_result_dic, orient='index')
save_df = bowler_stat_df.sort_values(by=["Wickets", "Economy"], ascending=[False, True])
save_df.to_csv("Evaluation/Evaluate_Bowler.csv")
save_df.head(20)

## Players Matchwise Evaluation 

In [ ]:
actual_batsmen_df = pd.read_csv("Evaluation/Actual_Batsmen.csv")
evaluate_batsmen_df = pd.read_csv("Evaluation/Evaluate_Batsmen.csv")
actual_bowler_df = pd.read_csv("Evaluation/Actual_Bowler.csv")
evaluate_bowler_df = pd.read_csv("Evaluation/Evaluate_Bowler.csv")

In [ ]:
for player in list(actual_batsmen_df["Unnamed: 0"])[:5]: 
    if player not in evaluator.batsmen_stat:
        continue
    player_actual = [i["Runs"] for i in actualstat.batsmen_stat[player]]
    player_evaluator = [i["Runs"] for i in evaluator.batsmen_stat[player]]
    plt.hist([player_evaluator, player_actual], 
            density=True, bins=10,
            label=["Our Simulation", "Actual Matches"])
    plt.legend()
    plt.xlabel(f"{player} runs in a match", fontweight='bold')
    plt.ylabel("Relative Frequency", fontweight='bold')
    fig = plt.gcf()
    fig.subplots_adjust(bottom=0.15)
    fig.subplots_adjust(left=0.15)
    fig.set_size_inches(10, 5)
    plt.savefig(f"Images\\Evaluation\\runs_histogram_{'_'.join(player.split())}.jpeg", dpi=300)
    plt.show()

In [ ]:
for player in list(actual_bowler_df["Unnamed: 0"])[:6]: 
    if player not in evaluator.bowler_stat:
        continue
    player_actual = [i["Wickets Taken"] for i in actualstat.bowler_stat[player]]
    player_evaluator = [i["Wickets Taken"] for i in evaluator.bowler_stat[player]]
    plt.hist([player_evaluator, player_actual], 
            density=True, bins=10,
            label=["Our Simulation", "Actual Matches"])
    plt.legend()
    plt.xlabel(f"{player} wickets in a match", fontweight='bold')
    plt.ylabel("Relative Frequency", fontweight='bold')
    fig = plt.gcf()
    fig.subplots_adjust(bottom=0.15)
    fig.subplots_adjust(left=0.15)
    fig.set_size_inches(10, 5)
    plt.savefig(f"Images\\Evaluation\\wickets_histogram_{'_'.join(player.split())}.jpeg", dpi=300)
    plt.show()